In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json, to_date, col, to_utc_timestamp, explode, split
from pyspark.sql.types import LongType, StructType, StringType
from pyspark import SparkContext, SparkConf
from datetime import datetime
from tweet_parser import TweetParser
import yaml
import configparser
import os
import pytz

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell'

config = ""
with open('../secrets.yml', 'r') as file:
    config = yaml.safe_load(file)

s3_conf = configparser.ConfigParser()
config_path = os.path.join(os.path.expanduser('~'), '.aws/credentials')
s3_conf.read(config_path)


#spark configuration
conf = SparkConf() \
    .set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true') \
    .set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true') \
    .setAppName('Wordle score streaming').setMaster('local[*]')

sc=SparkContext(conf=conf)
sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')
sc.setLogLevel("ERROR")

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', s3_conf.get('development', 'aws_access_key_id'))
hadoopConf.set('fs.s3a.secret.key', s3_conf.get('development', 'aws_secret_access_key'))
hadoopConf.set('fs.s3a.endpoint', 's3-eu-central-1.amazonaws.com')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

def getResults(text):
    return TweetParser(text).wordle_result_exist()

## Converting date string format
def getDate(x):
    if x is not None:
        return str(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

## UDF declaration
date_fn = udf(getDate, StringType())
attempts_fn = udf(lambda  x: getResults(x), StringType())

spark=SparkSession(sc)

lines = spark \
    .readStream \
    .format("kinesis") \
    .option("streamName", config['streamName']) \
    .option("endpointUrl", "https://kinesis.eu-central-1.amazonaws.com") \
    .option("awsAccessKeyId", s3_conf.get('development', 'aws_access_key_id')) \
    .option("awsSecretKey", s3_conf.get('development', 'aws_secret_access_key')) \
    .option("initialPosition", "earliest") \
    .load()
# .option("startingposition", "TRIM_HORIZON") \


schema = StructType(). \
    add('id', LongType(), False). \
    add('created_at', StringType(), False) .\
    add('user', StructType().add("id_str",StringType(), False), False). \
    add('text', StringType(), False)

filtered_data = lines \
    .selectExpr('CAST(data AS STRING)') \
    .select(from_json('data', schema).alias('tweet_data')) \
    .selectExpr('tweet_data.id', 'tweet_data.created_at', 'tweet_data.user.id_str AS user_id', 'tweet_data.text AS message') \
    .withColumn("created_at", to_utc_timestamp(date_fn("created_at"),"UTC")) \
    .withColumn('results', attempts_fn(col('message')))


filtered_data = filtered_data.filter(col('results') != "false")
filtered_data.printSchema()


# Write to s3
query = filtered_data \
    .writeStream \
    .format("parquet") \
    .partitionBy("created_at") \
    .option("path", "s3a://" +config['bucket'] + "/tweet_dump2/") \
    .option("checkpointLocation", "s3a://" + config['bucket'] + "/tweet_dump2/") \
    .start()


query.awaitTermination()